## initial code for our Hand Gesture Recognition

In [271]:
# ML models imports
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from skimage.feature import hog, local_binary_pattern

from skimage.filters import try_all_threshold

# destribution models imports
from scipy.stats import randint, uniform

# Data manipulation imports
import numpy as np
import pandas as pd
import tqdm as tqdm

# visualisation models imports
import matplotlib.pyplot as plt

# image processing imports
import skimage.io as io
import cv2
from skimage.transform import resize

# dealing with files
import os

# visual dataset (to test randomized gridsearch not needed for now)
from sklearn.datasets import make_hastie_10_2  # to test our models

# from utils import prepareData, LoadData, FeatureExtraction, preprocess
import csv

In [272]:
def segment(image):
    blured_image = cv2.GaussianBlur(image, (7, 7), 0)
    
    # Extract the Cr channel
    ycbcr_image = cv2.cvtColor(blured_image, cv2.COLOR_BGR2YCrCb)
    cr_channel = ycbcr_image[:,:,1]

    # Apply thresholding to obtain a binary image
    _, binary_img = cv2.threshold(cr_channel,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Define the structuring element for the closing operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    # Perform the closing operation
    closed_img = cv2.dilate(binary_img, kernel, iterations=1)

    # Find the contours in the binary image
    contours, hierarchy = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segmented_image = closed_img.copy()
    cv2.drawContours(segmented_image, contours, -1, 255, -1)
    
    # preprocessed = cv2.bitwise_and(image, image, mask=segmented_image)
    # preprocessed = cv2.cvtColor(preprocessed, cv2.COLOR_BGR2GRAY)

    return segmented_image, contours

In [273]:
from scipy.signal import find_peaks

def count_peaks(image, contours):
    M = cv2.moments(image)
    
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    centroid = np.array([cX, cY])

    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    max_contour = np.squeeze(max_contour)
    # Find peaks in the x-coordinate of contour points
    x = max_contour[:, 0]
    
    m, n = image.shape
    if(centroid[0] < n/2): # the hand is pointing to right
        peak_indices, _ = find_peaks(x, distance=200)
    else:  # the hand is pointing to left
        peak_indices, _ = find_peaks(-x, distance=200)
    
    peaks = max_contour[peak_indices]
    
    if len(peak_indices) == 0: return 0

    distance = np.linalg.norm(peaks - centroid, axis=1)
    max_peak_distance = np.max(distance)

    significant_peaks = peaks[distance >= 0.75*max_peak_distance]

    # Draw circles around peak points on the original image
    # for peak in peaks:
    #     cv2.circle(image, peak, 50, (0, 255, 0), -1)
    
    return(len(significant_peaks))

def ratio(contours):
    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    max_contour = np.squeeze(max_contour)

    area = cv2.contourArea(max_contour)
    x, y, w, h = cv2.boundingRect(max_contour)
    area_bounding_rectangle = w*h
    ratio = area/area_bounding_rectangle
    return ratio

In [68]:
def try_all(img, sigma=7, sec=25, block_size=11, threshold=cv2.ADAPTIVE_THRESH_GAUSSIAN_C):
    blured_image = cv2.GaussianBlur(img, (sigma, sigma), 0)
    
    gray = cv2.cvtColor(blured_image, cv2.COLOR_BGR2GRAY)

    # gray = cv2.cvtColor(blured_image, cv2.COLOR_BGR2GRAY)
    binary_img = cv2.adaptiveThreshold(gray, 255, threshold, cv2.THRESH_BINARY_INV, block_size, 2)
    
    
    # Define the structuring element for the closing operation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (sec, sec))
    # Perform the closing operation
    closed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)
    
    contours, hierarchy = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segmented_image = closed_img.copy()
    cv2.drawContours(segmented_image, contours, -1, 255, -1)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sec, sec))
    segmented_image = cv2.morphologyEx(segmented_image, cv2.MORPH_CLOSE, kernel)
    
    return segmented_image

In [288]:
def FeatureExtraction(image):
        
    # Extract the hog features
    # block_norm uses L2 norm with hysterisis for reducing effect of illuminacity
    # transform_sqrt for applying gamma correction
    preprocessed_image, contours = segment(image)
    
    resized_image = resize(preprocessed_image,(64,128))

    hog_features = hog(resized_image, block_norm='L2-Hys', feature_vector=True, transform_sqrt=True, pixels_per_cell=(12, 12), cells_per_block=(2, 2))
    contour_to_ROI = ratio(contours)
    peak_num = count_peaks(preprocessed_image, contours)
    
    features = np.append(hog_features, (contour_to_ROI, peak_num))

    return features

In [289]:
def LoadData():
    Features=[]
    labels=[]

    i = 0

    
    for gender in ["men","Women"]:
        datadir = r"Dataset\{}".format(gender)
        # loop over gender
        for hand in os.listdir(datadir): 
            # loop over each class [0,1,2,3,4,5]
            for img in os.listdir(datadir+ "/" +str(hand)):
                # ignoring anything except images
                if((img.split('.')[-1]).lower() not in ['jpg','png','jpeg']):
                    continue

                # loading our images
                img_array=io.imread(datadir + "/" + str(hand) + "/" + img )  # approx 2500 * 4000

                # append extracted features to Featurees list   
                Feature = FeatureExtraction(img_array)        
                Features.append(Feature) 

                # append class of image.
                labels.append(hand)  

                print(f'image Number: {i}')
                i+=1
                # # print(f'saving block : {i//100}')
                # # np.save(f'Features/{i}.npy', Feature)
                # # Features = []
                # # labels = []
                # with open("Features.csv", 'a') as csvfile:
                #     csvwriter = csv.writer(csvfile)
                #     csvwriter.writerow(Feature)

    return np.asarray(Features),np.asarray(labels)

In [ ]:
Features, labels = LoadData() 

* # Selecting the best model

- ### Define hyperparameter grids for each model

In [291]:
param_distributions = {
    'RandomForestClassifier': {
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'GradientBoostingClassifier': {
        'learning_rate': uniform(0.01, 0.2),
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'SVC': {
        'C': uniform(0.01, 10),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree': randint(2, 5),
        'gamma': ['scale', 'auto'] + list(np.arange(0.1, 1, 0.1))
    },
    'LogisticRegression': {
        'C': uniform(0.01, 10),
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': randint(50, 500)
    },
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'KNeighborsClassifier': {
        'n_neighbors': randint(3, 30),
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'leaf_size': randint(10, 100)
    },
    'GaussianNB': {
        'var_smoothing': uniform(1e-09, 1e-07)
    },
    'MLPClassifier': {
        'hidden_layer_sizes': [(50, 50), (100,), (100, 50)],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': uniform(0.0001, 0.01),
        'max_iter': randint(100, 1000)
    }
}

- ### Create a list of models to train (as example)

In [292]:
models = [
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    SVC(),
    KNeighborsClassifier(),
    GaussianNB()
]

- ### Loop over the models and fit  

In [293]:
# load our dummy data to test the randomizedSearche function
x,y = Features,labels
df = pd.DataFrame(x)
df['Y'] = y

train, test = train_test_split(df, test_size=0.2) # this function shuffles the data points, and splits the data into
                                                  # 80% training set and 20% test set (indicated by test_size=0.2)
X_train, Y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, Y_test = test.iloc[:, :-1], test.iloc[:, -1]

In [ ]:
for i, model in enumerate(models):
    print(f'Training Model {i+1}/{len(models)}: {str(model)[:-2]}')
    # Define randomized grid search
    random_search = RandomizedSearchCV(model, param_distributions[str(model)[:-2]], n_iter=10,cv=5, n_jobs=-1) # n_jobs means number of jobs to run in parallel. None means 1,
                                                                                                                # -1 means using all processors 😈.
    # Fit the randomized grid search to the data
    random_search.fit(X_train, Y_train)
    print(f'Best score: {random_search.best_score_:.3f}')
    print(f'Best parameters: {random_search.best_params_}\n')